In [1]:
import nltk
import re
import numpy as np # linear algebra
import pandas as pd # data processing
import random
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import SnowballStemmer
stemmer=SnowballStemmer('english')

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize

In [7]:
df=pd.read_csv('Amazon_Unlocked_Mobile.csv')
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [9]:
df=df[['Reviews','Rating']]

In [10]:
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413778 entries, 0 to 413839
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Reviews  413778 non-null  object
 1   Rating   413778 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 9.5+ MB


In [11]:
df=df[df['Rating']!=3]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382015 entries, 0 to 413839
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Reviews  382015 non-null  object
 1   Rating   382015 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 8.7+ MB


In [12]:
df=df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382015 entries, 0 to 382014
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Reviews  382015 non-null  object
 1   Rating   382015 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ MB


In [13]:
df['sentiment']=np.where(df['Rating'] > 3, 1, 0)
df.head()

,Reviews,Rating,sentiment
0,I feel so LUCKY to have found this used (phone...,5,1
1,"nice phone, nice up grade from my pantach revu...",4,1
2,Very pleased,5,1
3,It works good but it goes slow sometimes but i...,4,1
4,Great phone to replace my lost phone. The only...,4,1


In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/datalore/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
Cstopwords=set(stopwords.words('english')+list(punctuation))
from nltk.stem import WordNetLemmatizer
lemma=WordNetLemmatizer()

In [19]:
def clean_review(review_column):
    review_corpus=[]
    for i in range(0,len(review_column)):
        review=review_column[i]
        #review=BeautifulSoup(review,'lxml').text
        review=re.sub('[^a-zA-Z]',' ',review)
        review=str(review).lower()
        review=word_tokenize(review)
        #review=[stemmer.stem(w) for w in review if w not in Cstopwords]
        review=[lemma.lemmatize(w) for w in review ]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [24]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/datalore/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/datalore/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [25]:
review_column=df['Reviews']
review_corpus=clean_review(review_column)

In [26]:
df['clean_review']=review_corpus
df.tail(20)

,Reviews,Rating,sentiment,clean_review
381995,"This phone is simple, very good , and it works...",5,1,this phone is simple very good and it work exc...
381996,Good sturdy phone for a pre-teen to have avail...,4,1,good sturdy phone for a pre teen to have avail...
381997,This is the second junk Convoy I have owned. T...,1,0,this is the second junk convoy i have owned th...
381998,I BOUGHT THIS PHONE FOR MY HUSBAND AND HE LOVE...,5,1,i bought this phone for my husband and he love...
381999,They said phone was normal wear but it was a l...,1,0,they said phone wa normal wear but it wa a lie...
382000,"You could shoot this out of a potato gun, and ...",5,1,you could shoot this out of a potato gun and p...
382001,Bought this for my mother and she loves it. Gr...,5,1,bought this for my mother and she love it grea...
382002,"Excellent phone, as advertised. Love the push-...",5,1,excellent phone a advertised love the push to ...
382003,works great and picks up signal in place my ot...,4,1,work great and pick up signal in place my othe...
382004,"Great phone. Large keys, best flip phone I hav...",5,1,great phone large key best flip phone i have o...


In [27]:
from sklearn.feature_extraction.text import CountVectorizer

In [29]:
cv=CountVectorizer(max_features=20000,min_df=5,ngram_range=(1,2))

In [30]:
X1=cv.fit_transform(df['clean_review'])
X1.shape

(382015, 20000)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
tfidf=TfidfVectorizer(min_df=5, max_df=0.95, max_features = 20000, ngram_range = ( 1, 2 ),
                              sublinear_tf = True)

In [33]:
tfidf=tfidf.fit(df['clean_review'])

In [34]:
X2=tfidf.transform(df['clean_review'])
X2.shape

(382015, 20000)

In [35]:
y=df['sentiment'].values
y.shape

(382015,)

In [37]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(305612, 20000) (305612,)
(76403, 20000) (76403,)


In [38]:
# average positive reviews in train and test
print('mean positive review in train : {0:.3f}'.format(np.mean(y_train)))
print('mean positive review in test : {0:.3f}'.format(np.mean(y_test)))

mean positive review in train : 0.746
mean positive review in test : 0.745


In [39]:
from sklearn.linear_model import LogisticRegression as lr

In [40]:
model_lr=lr(random_state=0)

In [41]:
%%time
model_lr=lr(penalty='l2',C=1.0,random_state=0)
model_lr.fit(X_train,y_train)
y_pred_lr=model_lr.predict(X_test)
print('accuracy for Logistic Regression :',accuracy_score(y_test,y_pred_lr))
print('confusion matrix for Logistic Regression:\n',confusion_matrix(y_test,y_pred_lr))
print('F1 score for Logistic Regression :',f1_score(y_test,y_pred_lr))
print('Precision score for Logistic Regression :',precision_score(y_test,y_pred_lr))
print('recall score for Logistic Regression :',recall_score(y_test,y_pred_lr))
print('AUC: ', roc_auc_score(y_test, y_pred_lr))

accuracy for Logistic Regression : 0.963273693441357
confusion matrix for Logistic Regression:
 [[17786  1679]
 [ 1127 55811]]
F1 score for Logistic Regression : 0.9754780298528333
Precision score for Logistic Regression : 0.970794920855801
recall score for Logistic Regression : 0.9802065404475043
AUC:  0.9469745776987073
CPU times: user 15.1 s, sys: 7.18 s, total: 22.3 s
Wall time: 11.6 s


/opt/python/envs/default/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
# get the feature names as numpy array
feature_names = np.array(cv.get_feature_names())

# Sort the coefficients from the model
sorted_coef_index = model_lr.coef_[0].argsort()

# Find the 10 smallest and 10 largest coefficients
# The 10 largest coefficients are being indexed using [:-11:-1] 
# so the list returned is in order of largest to smallest
print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['not happy' 'not satisfied' 'no good' 'not worth' 'worst' 'garbage'
 'awful' 'junk' 'not good' 'not very']

Largest Coefs: 
['excelent' 'excelente' 'exelente' 'not bad' 'excellent' 'no issue'
 'perfect' 'perfecto' 'awesome' 'great']


In [45]:
from sklearn.naive_bayes import MultinomialNB
model_nb=MultinomialNB()
model_nb.fit(X_train,y_train)
y_pred_nb=model_nb.predict(X_test)
print('accuracy for Naive Bayes Classifier :',accuracy_score(y_test,y_pred_nb))
print('confusion matrix for Naive Bayes Classifier:\n',confusion_matrix(y_test,y_pred_nb))
print('F1 score for Logistic Regression :',f1_score(y_test,y_pred_nb))
print('Precision score for Logistic Regression :',precision_score(y_test,y_pred_nb))
print('recall score for Logistic Regression :',recall_score(y_test,y_pred_nb))
print('AUC: ', roc_auc_score(y_test, y_pred_nb))

accuracy for Naive Bayes Classifier : 0.93337957933589
confusion matrix for Naive Bayes Classifier:
 [[17047  2418]
 [ 2672 54266]]
F1 score for Logistic Regression : 0.9552023375754695
Precision score for Logistic Regression : 0.957342459953426
recall score for Logistic Regression : 0.9530717622677298
AUC:  0.9144243989864208


In [46]:
# get the feature names as numpy array
feature_names = np.array(cv.get_feature_names())

# Sort the coefficients from the model
sorted_coef_index = model_nb.coef_[0].argsort()

# Find the 10 smallest and 10 largest coefficients
# The 10 largest coefficients are being indexed using [:-11:-1] 
# so the list returned is in order of largest to smallest
print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['worst purchase' 'nothing work' 'zero star' 'reported stolen'
 'never recommend' 'royalty' 'total waste' 'or refund' 'is scam'
 'went completely']

Largest Coefs: 
['the' 'it' 'phone' 'and' 'to' 'is' 'this' 'for' 'my' 'with']


/opt/python/envs/default/lib/python3.8/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute coef_ was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


In [47]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
%%time
model_rf=RandomForestClassifier()
model_rf.fit(X_train,y_train)
y_pred_rf=model_rf.predict(X_test)
print('accuracy for Random Forest Classifier :',accuracy_score(y_test,y_pred_rf))
print('confusion matrix for Random Forest Classifier:\n',confusion_matrix(y_test,y_pred_rf))

accuracy for Random Forest Classifier : 0.9767941049435231
confusion matrix for Random Forest Classifier:
 [[18245  1220]
 [  553 56385]]
CPU times: user 48min 17s, sys: 3.97 s, total: 48min 21s
Wall time: 48min 20s


In [49]:
# get the feature names as numpy array
feature_names = np.array(cv.get_feature_names())

# Sort the coefficients from the model
sorted_coef_index = model_rf.feature_importances_.argsort()

# Find the 10 smallest and 10 largest coefficients
# The 10 largest coefficients are being indexed using [:-11:-1] 
# so the list returned is in order of largest to smallest
print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['safari' 'not tested' 'image stabilization' 'pixel density'
 'watching youtube' 'smooth the' 'very crisp' 'with update' 'me away'
 'toggle']

Largest Coefs: 
['not' 'great' 'love' 'return' 'the' 'bad' 'good' 'excellent' 'after'
 'month']
